# * VINSIGHT : CEO Report

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100GEO' --Total Revenue (Geo)
        , 'B1R000100' --Prepaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB4S000500' --TVS Active Subs
         
        , 'B1S000101CS' --Prepaid Gross Adds
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB4S001400CS' --TVS Now Gross Adds
         
        , 'B1S000200' --Prepaid Churn Subs
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-06, 09:29:25

DataFrame: 12528 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [4]:
# chk_src_df.tail()

## Period Parameter

In [5]:
''' Parameter '''

# v_tm_key_qtr = curr_qtr
v_tm_key_qtr = 20241

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202501

# v_tm_key_wk = curr_wk
v_tm_key_wk = last_3_wk
# v_tm_key_wk = 2025020

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

print(f'v_tm_key_qtr: {v_tm_key_qtr}')
print(f'v_tm_key_mth: {v_tm_key_mth}')
print(f'v_tm_key_wk: {v_tm_key_wk}')
print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_qtr: 20241
v_tm_key_mth: 202505
v_tm_key_wk: 2025020
v_tm_key_mth: 202505


## Revenue(Geo) by Period

### Yearly

In [6]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df[col] = rev_geo_yearly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df

,TM_KEY_YR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2024,2025-06-06 03:26:41,"122,436,333,223","48,319,863,915","51,270,975,737","18,827,249,425","4,018,244,147"
1,2025,2025-06-06 03:26:41,"60,110,076,335","20,376,173,103","29,057,470,213","8,251,333,315","1,478,150,211"


### Quarterly

In [7]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df[col] = rev_geo_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df

,TM_KEY_QTR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,20241,2025-06-06 03:26:41,"25,511,481,753","11,842,176,786","8,086,172,296","4,569,920,004","1,013,212,668"
1,20242,2025-06-06 03:26:41,"25,446,081,657","11,858,030,293","7,953,686,559","4,665,781,597","968,583,208"
2,20243,2025-06-06 03:26:41,"35,706,816,037","12,164,985,004","17,628,033,120","4,803,785,483","1,110,012,430"
3,20244,2025-06-06 03:26:41,"35,771,953,777","12,454,671,833","17,603,083,763","4,787,762,341","926,435,841"
4,20251,2025-06-06 03:26:41,"35,649,449,169","12,375,362,677","17,580,761,024","4,810,187,607","883,137,861"
5,20252,2025-06-06 03:26:41,"24,460,627,166","8,000,810,426","11,476,709,189","3,441,145,708","595,012,350"


### Monthly

In [8]:
''' Revenue(Geo) Monthly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df[col] = rev_geo_monthly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df

,TM_KEY_MTH,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202505,2025-06-06 03:26:41,"11,870,473,364","3,927,715,673","5,574,765,425","1,614,210,491","291,207,750"
1,202506,2025-06-06 03:26:41,"698,568,595",0,0,"214,193,128",0


### Weekly

In [9]:
''' Revenue(Geo) Weekly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]
rev_geo_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = rev_geo_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df[col] = rev_geo_weekly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2025020,20250518,2025-06-06 03:26:41,"2,590,493,281","911,912,529","1,292,785,237","341,903,573","43,891,942"
1,2025021,20250525,2025-06-06 03:26:41,"3,039,704,384","885,491,358","1,595,383,526","497,345,069","61,484,431"
2,2025022,20250601,2025-06-06 03:26:41,"1,978,092,854","528,899,126","604,545,071","174,220,946","113,165,413"
3,2025023,20250605,2025-06-06 03:26:41,"603,880,325",0,0,"214,193,128",0


### Daily
    The information is valid until 28 May 2025

In [26]:
''' Revenue(Geo) Daily '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df[col] = rev_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,"81,089,268",0,0,0,0
2,202506,20250603,2025-06-06 03:26:41,0,0,0,0,0
3,202506,20250602,2025-06-06 03:26:41,"522,791,057",0,0,"214,193,128",0
4,202506,20250601,2025-06-06 03:26:41,"94,688,270",0,0,0,0
5,202505,20250531,2025-06-06 03:26:41,"190,274,144",0,0,0,"91,335,082"
6,202505,20250530,2025-06-06 03:26:41,"96,348,511",0,0,0,0
7,202505,20250529,2025-06-06 03:26:41,"400,256,266","132,969,814",0,0,0
8,202505,20250528,2025-06-06 03:26:41,"761,888,575","136,386,168","429,451,129","174,220,946","21,830,332"
9,202505,20250527,2025-06-06 03:26:41,"217,062,705","129,374,833","87,687,873",0,0


## Subscriber by Period

### Yearly

In [11]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['FTTX_RPT'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df[col] = sub_yearly_df[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2024,2025-06-06 03:26:41,"28,134,783","14,158,698","3,154,889","1,180,410"
1,2025,2025-06-06 03:26:41,"26,546,876","14,066,372","3,198,020","1,127,338"


### Quarterly

In [12]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['FTTX_RPT'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df[col] = sub_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,20241,2025-06-06 03:26:41,0,0,"3,090,287","1,258,065"
1,20242,2025-06-06 03:26:41,"27,835,150","14,226,167","3,107,084","1,236,606"
2,20243,2025-06-06 03:26:41,"27,576,181","14,186,615","3,127,304","1,205,116"
3,20244,2025-06-06 03:26:41,"28,134,783","14,158,698","3,154,889","1,180,410"
4,20251,2025-06-06 03:26:41,"27,140,596","14,168,597","3,187,717","1,153,137"
5,20252,2025-06-06 03:26:41,"26,546,876","14,066,372","3,198,020","1,127,338"


### Monthly

In [13]:
''' Subscriber Monthly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['FTTX_RPT'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df[col] = sub_monthly_df[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202505,2025-06-06 03:26:41,"26,496,116","14,160,783","3,203,127","1,129,379"
1,202506,2025-06-06 03:26:41,"26,546,876","14,066,372","3,198,020","1,127,338"


### Weekly

In [14]:
''' Subscriber Weekly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['FTTX_RPT'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]
sub_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = sub_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df[col] = sub_weekly_df[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2025020,20250518,2025-06-06 03:26:41,"26,631,443","14,177,262","3,191,938","1,134,057"
1,2025021,20250525,2025-06-06 03:26:41,"26,635,512","14,068,090","3,193,370","1,131,955"
2,2025022,20250601,2025-06-06 03:26:41,"26,439,246","14,162,131","3,205,472","1,128,753"
3,2025023,20250604,2025-06-06 03:26:41,"26,546,876","14,066,372","3,198,020","1,127,338"


### Daily

In [27]:
''' Subscriber Daily '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]
sub_daily_df = sub_daily_df.loc[sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['FTTX_RPT'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df[col] = sub_daily_df[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,0,0,0,"1,127,338"
2,202506,20250603,2025-06-06 03:26:41,"26,546,876","14,066,372","3,198,020","1,127,815"
3,202506,20250602,2025-06-06 03:26:41,"26,545,204","14,071,114","3,206,516","1,128,315"
4,202506,20250601,2025-06-06 03:26:41,"26,439,246","14,162,131","3,205,472","1,128,753"
5,202505,20250531,2025-06-06 03:26:41,"26,496,116","14,160,783","3,203,127","1,129,379"
6,202505,20250530,2025-06-06 03:26:41,"26,544,725","14,065,089","3,201,195","1,129,710"
7,202505,20250529,2025-06-06 03:26:41,"26,582,772","14,159,959","3,198,852","1,130,328"
8,202505,20250528,2025-06-06 03:26:41,"26,612,350","14,167,990","3,195,276","1,130,797"
9,202505,20250527,2025-06-06 03:26:41,"26,627,728","14,166,874","3,199,020","1,130,797"


## Gross Adds by Period

### Yearly

In [16]:
''' Gross Adds Yearly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE_GA'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_GA_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_GA_CONN_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW_GA'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df[col] = ga_yearly_df[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df

,TM_KEY_YR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2024,2025-06-06 03:26:41,"30,607,404","2,642,885","558,527","7,682"
1,2025,2025-06-06 03:26:41,"10,647,838","967,194","253,828","8,740"


### Quarterly

In [17]:
''' Gross Adds Quarterly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

ga_quarterly_df['PRE_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_GA_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_GA_CONN_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df[col] = ga_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,20241,2025-06-06 03:26:41,"8,953,743","701,773","130,958",0
1,20242,2025-06-06 03:26:41,"7,514,316","698,111","141,474",0
2,20243,2025-06-06 03:26:41,"7,156,127","662,059","147,285","4,992"
3,20244,2025-06-06 03:26:41,"6,983,218","580,942","138,810","2,690"
4,20251,2025-06-06 03:26:41,"6,673,218","569,340","145,692","3,337"
5,20252,2025-06-06 03:26:41,"3,974,620","397,854","108,136","5,403"


### Monthly

In [18]:
''' Gross Adds Monthly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_monthly_df['PRE_GA'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_GA_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_GA_CONN_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW_GA'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df[col] = ga_monthly_df[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202505,2025-06-06 03:26:41,"1,912,792","182,712","52,734","2,806"
1,202506,2025-06-06 03:26:41,"182,506","17,872","6,920",245


### Weekly

In [19]:
''' Gross Adds Weekly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

ga_weekly_df['PRE_GA'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_GA_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_GA_CONN_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW_GA'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]
ga_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = ga_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df[col] = ga_weekly_df[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2025020,20250518,2025-06-06 03:26:41,"424,641","43,160","12,285",419
1,2025021,20250525,2025-06-06 03:26:41,"407,397","36,559","11,961",563
2,2025022,20250601,2025-06-06 03:26:41,"429,033","36,581","11,562",837
3,2025023,20250605,2025-06-06 03:26:41,"117,892","12,301","5,251",162


### Daily

In [28]:
''' Gross Adds Daily '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_daily_df = ga_daily_df.loc[ga_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_daily_df['PRE_GA'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_GA_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_GA_CONN_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW_GA'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df[col] = ga_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,0,0,"1,822",0
2,202506,20250603,2025-06-06 03:26:41,"57,933","6,496","1,733",107
3,202506,20250602,2025-06-06 03:26:41,"59,959","5,805","1,696",55
4,202506,20250601,2025-06-06 03:26:41,"64,614","5,571","1,669",83
5,202505,20250531,2025-06-06 03:26:41,"71,873","5,697","1,696",144
6,202505,20250530,2025-06-06 03:26:41,"68,285","5,203","1,650",151
7,202505,20250529,2025-06-06 03:26:41,"61,793","4,821","1,572",146
8,202505,20250528,2025-06-06 03:26:41,"54,979","4,939","1,674",121
9,202505,20250527,2025-06-06 03:26:41,"53,237","5,542","1,613",110


## Churn Subs by Period

### Yearly

In [21]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_yearly_df = churn_yearly_df.loc[churn_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_yearly_df['PRE_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='B1S000200', churn_yearly_df['P'], 0)
churn_yearly_df['POST_CHURN_B2C'] = np.where(churn_yearly_df['METRIC_CD']=='B2S010200', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13100', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_VOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13102', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13103', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN(CF)'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13104', churn_yearly_df['P'], 0)
churn_yearly_df['TVS_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER14100', churn_yearly_df['P'], 0)

churn_yearly_df = churn_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_yearly_df = churn_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_yearly_df = churn_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_yearly_df[col] = churn_yearly_df[col].apply(lambda x: format(x, ',.0f'))
churn_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2024,2025-06-06 03:26:41,"34,112,179","2,857,216","456,668","170,154","281,222","282,188","194,250"
1,2025,2025-06-06 03:26:41,"32,820,581","960,270","166,779","66,411","147,988","149,595","90,734"


### Quarterly

In [22]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['METRIC_CD'].isin(v_metric_list)]
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

churn_quarterly_df['PRE_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='B1S000200', churn_quarterly_df['P'], 0)
churn_quarterly_df['POST_CHURN_B2C'] = np.where(churn_quarterly_df['METRIC_CD']=='B2S010200', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13100', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_VOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13102', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13103', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN(CF)'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13104', churn_quarterly_df['P'], 0)
churn_quarterly_df['TVS_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER14100', churn_quarterly_df['P'], 0)

churn_quarterly_df = churn_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_quarterly_df = churn_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_quarterly_df = churn_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_quarterly_df[col] = churn_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
churn_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,20241,2025-06-06 03:26:41,"9,491,554","871,395","118,742","43,223","72,945","73,317","48,913"
1,20242,2025-06-06 03:26:41,"8,344,376","718,051","114,261","43,677","70,807","70,972","40,970"
2,20243,2025-06-06 03:26:41,"8,899,074","664,232","118,872","44,269","67,909","68,985","48,795"
3,20244,2025-06-06 03:26:41,"7,377,175","603,538","104,793","38,985","69,561","68,914","55,572"
4,20251,2025-06-06 03:26:41,"7,728,433","553,494","104,815","38,083","68,600","68,746","47,799"
5,20252,2025-06-06 03:26:41,"25,092,148","406,776","61,964","28,328","79,388","80,849","42,935"


### Monthly

In [23]:
''' Churn Subs Monthly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['METRIC_CD'].isin(v_metric_list)]
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_monthly_df['PRE_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='B1S000200', churn_monthly_df['P'], 0)
churn_monthly_df['POST_CHURN_B2C'] = np.where(churn_monthly_df['METRIC_CD']=='B2S010200', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13100', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_VOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13102', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13103', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN(CF)'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13104', churn_monthly_df['P'], 0)
churn_monthly_df['TVS_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER14100', churn_monthly_df['P'], 0)

churn_monthly_df = churn_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_monthly_df = churn_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_monthly_df = churn_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_monthly_df[col] = churn_monthly_df[col].apply(lambda x: format(x, ',.0f'))
churn_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202505,2025-06-06 03:26:41,"22,348,738","211,802","38,488","14,562","56,303","57,851","22,325"
1,202506,2025-06-06 03:26:41,"346,104","20,895",-814,641,0,0,"3,279"


### Weekly

In [24]:
''' Churn Subs Weekly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['METRIC_CD'].isin(v_metric_list)]
churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

churn_weekly_df['PRE_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='B1S000200', churn_weekly_df['P'], 0)
churn_weekly_df['POST_CHURN_B2C'] = np.where(churn_weekly_df['METRIC_CD']=='B2S010200', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13100', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_VOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13102', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13103', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN(CF)'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13104', churn_weekly_df['P'], 0)
churn_weekly_df['TVS_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER14100', churn_weekly_df['P'], 0)

churn_weekly_df = churn_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_weekly_df = churn_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_weekly_df = churn_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]
churn_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = churn_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_weekly_df[col] = churn_weekly_df[col].apply(lambda x: format(x, ',.0f'))
churn_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2025020,20250518,2025-06-06 03:26:41,"519,290","50,095","8,076","3,232","14,649",0,"5,086"
1,2025021,20250525,2025-06-06 03:26:41,"541,806","49,489","9,725","2,595","27,982",0,"4,611"
2,2025022,20250601,2025-06-06 03:26:41,"20,542,155","40,087","-1,047","3,495","5,987",0,"5,354"
3,2025023,20250605,2025-06-06 03:26:41,"278,435","16,596",0,0,0,0,"2,137"


### Daily

In [29]:
''' Churn Subs Daily '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_daily_df = churn_daily_df.loc[churn_daily_df['METRIC_CD'].isin(v_metric_list)]
churn_daily_df = churn_daily_df.loc[churn_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_daily_df['PRE_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='B1S000200', churn_daily_df['P'], 0)
churn_daily_df['POST_CHURN_B2C'] = np.where(churn_daily_df['METRIC_CD']=='B2S010200', churn_daily_df['P'], 0)
churn_daily_df['FTTX_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13100', churn_daily_df['P'], 0)
churn_daily_df['FTTX_VOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13102', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13103', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN(CF)'] = np.where(churn_daily_df['METRIC_CD']=='TSER13104', churn_daily_df['P'], 0)
churn_daily_df['TVS_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER14100', churn_daily_df['P'], 0)

churn_daily_df = churn_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_daily_df = churn_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_daily_df = churn_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_daily_df[col] = churn_daily_df[col].apply(lambda x: format(x, ',.0f'))
churn_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,"123,185",0,0,0,0,0,0
2,202506,20250603,2025-06-06 03:26:41,"78,444","1,928",0,0,0,0,"1,113"
3,202506,20250602,2025-06-06 03:26:41,"76,806","14,668",0,0,0,0,"1,024"
4,202506,20250601,2025-06-06 03:26:41,"67,669","4,299",-814,641,0,0,"1,142"
5,202505,20250531,2025-06-06 03:26:41,"64,187","2,599",-355,600,31,0,"1,102"
6,202505,20250530,2025-06-06 03:26:41,"20,154,633","4,794",-805,490,1,0,852
7,202505,20250529,2025-06-06 03:26:41,"73,586","15,380","-2,084",428,"3,077",0,83
8,202505,20250528,2025-06-06 03:26:41,"61,945","3,790","5,363",415,0,0,567
9,202505,20250527,2025-06-06 03:26:41,"60,307","4,721","-1,936",457,"2,878",0,783
